In [2]:
import os
import sys
# sys.path.append("../virtuosoNet")

from pathlib import Path
import pandas as pd
import music21 as m21
import pretty_midi as pm
from  matplotlib import pyplot as plt
import util as u

%matplotlib inline
import numpy as np
import math
import pandas as pd
import librosa
import soundfile

from subprocess import Popen, PIPE

import IPython.display as ipd

from collections import Counter

In [3]:
BASE_PATH = "../"

### Produce the midi score from the xml using musescore

### Produce the annotations from the midi score

In [4]:
def quant_downbeat_to_annotations(midi_quant_path):
    mididata = pm.PrettyMIDI(str(midi_quant_path))
    downbeats = mididata.get_downbeats()
    #the downbeat duration (in theory)
    db_len = downbeats[2]-downbeats[1]
    downbeat = {}
    for i, db in enumerate(downbeats[:-1]):
        assert(db not in downbeat) #check if all values are uniques
        if math.isclose(downbeats[i+1]-downbeats[i], db_len, rel_tol=1e-2):
            downbeat[db] = "db"
        else:
            downbeat[db]= "dbW"
    #sort the dictionary just in case there are weird values
    ord_downbeat = {k: v for k, v in sorted(downbeat.items(), key=lambda item: item[1])}
    return ord_downbeat

def quant_downbeat_to_annotations_from_midi(mididata):
    downbeats = mididata.get_downbeats()
    #the downbeat duration (in theory)
    db_len = downbeats[2]-downbeats[1]
    downbeat = {}
    for i, db in enumerate(downbeats[:-1]):
        assert(db not in downbeat) #check if all values are uniques
        if math.isclose(downbeats[i+1]-downbeats[i], db_len, rel_tol=1e-2):
            downbeat[db] = "db"
        else:
            downbeat[db]= "dbW"
    #sort the dictionary just in case there are weird values
    ord_downbeat = {k: v for k, v in sorted(downbeat.items(), key=lambda item: item[1])}
    return ord_downbeat

def quant_beat_to_annotations(midi_quant_path):
    mididata = pm.PrettyMIDI(str(midi_quant_path))
    beats = mididata.get_beats()
    #the beat duration (in theory)
    b_len = beats[40]-beats[39] #multiple short beats in the beginning sometimes
    beat = {}
    for i, b in enumerate(beats[:-1]):
        assert(b not in beat) #check if all values are uniques
        if math.isclose(beats[i+1]-beats[i], b_len, rel_tol=1e-2):
            beat[b] = "b"
        else:
            beat[b] = "bW"
    #sort the dictionary just in case there are weird values
    ord_beat = {k: v for k, v in sorted(beat.items(), key=lambda item: item[1])}
    return beat

def quant_beat_to_annotations_from_midi(mididata):
    beats = mididata.get_beats()
    #the beat duration (in theory)
    b_len = beats[40]-beats[39] #multiple short beats in the beginning sometimes
    beat = {}
    for i, b in enumerate(beats[:-1]):
        assert(b not in beat) #check if all values are uniques
        if math.isclose(beats[i+1]-beats[i], b_len, rel_tol=1e-2):
            beat[b] = "b"
        else:
            beat[b] = "bW"
    #sort the dictionary just in case there are weird values
    ord_beat = {k: v for k, v in sorted(beat.items(), key=lambda item: item[1])}
    return beat

def aggregate_annotations(beat, downbeat):
    #merge the downbeat and the beat, with downbeat overwriting beat informations
    annotations ={**beat, **downbeat}
    return annotations

def beat_downbeat_to_text(annotations,out_path):
    path = Path(out_path)
    #produce the txt
    file_content = ""
    for a in annotations:
        file_content+= str(a) + "\t" + str(a) + "\t" + annotations[a] + "\n"
    with open(out_path,"w") as f:
        f.write(file_content)
        
def annotations_to_audio(midi_quant_path, annotations, out_path):
    midi_data = pm.PrettyMIDI(str(midi_quant_path))
    audio_fs = 22100
    beat = [a for a in annotations if annotations[a][0] == "b"]
    downbeat = [a for a in annotations if annotations[a][0] == "d"]
    midi_audio = midi_data.synthesize(fs=audio_fs)
    audio_beat = librosa.clicks(beat, sr=audio_fs, click_freq=900, length=len(midi_audio))
    audio_downbeat = librosa.clicks(downbeat, sr=audio_fs, click_freq=1200, length=len(midi_audio))
    soundfile.write(out_path,audio_beat + audio_downbeat+ midi_audio,audio_fs)

In [5]:
all_df = pd.read_csv(Path(BASE_PATH,"metadata.csv"))
quant_df = all_df.drop_duplicates(subset='xml_score', keep='first')
quant_df[quant_df.midi_score == "Schubert/Wanderer_fantasie/midi_score.mid"]

,composer,title,folder,xml_score,midi_score,midi_performance,performance_annotations,midi_score_annotations,maestro_midi_performance,maestro_audio_performance,start,end,audio_performance
1024,Schubert,Wanderer_fantasie,Schubert/Wanderer_fantasie,Schubert/Wanderer_fantasie/xml_score.musicxml,Schubert/Wanderer_fantasie/midi_score.mid,Schubert/Wanderer_fantasie/Kolessova02.mid,Schubert/Wanderer_fantasie/Kolessova02_annotat...,Schubert/Wanderer_fantasie/midi_score_annotati...,NaN,NaN,NaN,NaN,NaN


In [6]:
row = quant_df.loc[1024]
row

composer                                                              Schubert
title                                                        Wanderer_fantasie
folder                                              Schubert/Wanderer_fantasie
xml_score                        Schubert/Wanderer_fantasie/xml_score.musicxml
midi_score                           Schubert/Wanderer_fantasie/midi_score.mid
midi_performance                    Schubert/Wanderer_fantasie/Kolessova02.mid
performance_annotations      Schubert/Wanderer_fantasie/Kolessova02_annotat...
midi_score_annotations       Schubert/Wanderer_fantasie/midi_score_annotati...
maestro_midi_performance                                                   NaN
maestro_audio_performance                                                  NaN
start                                                                      NaN
end                                                                        NaN
audio_performance                                   

In [7]:
score_path = row["xml_score"]
midi_quant_path = row["midi_score"]

print("processing score", score_path)

midi_data = pm.PrettyMIDI(str(Path(BASE_PATH,midi_quant_path)))      
        
dbs = quant_downbeat_to_annotations_from_midi(midi_data)
bs = quant_beat_to_annotations_from_midi(midi_data)
anns = aggregate_annotations(bs,dbs)


beat_downbeat_to_text(anns, str(Path(BASE_PATH,row["midi_score_annotations"])))

processing score Schubert/Wanderer_fantasie/xml_score.musicxml


/home/fosfrancesco/miniconda3/envs/mir_research/lib/python3.8/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


### Add the ts changes

In [8]:
# create the time signature changes

def score2ts_changes(score_path, remove_duplicates = True):
    score = m21.converter.parse(score_path)
    ts_changes = [ str(ts.numerator)+"/"+str(ts.denominator) for ts in score.parts[0].recurse().getElementsByClass(m21.meter.TimeSignature)]
    if remove_duplicates:
        #remove the duplicates
        ts_changes = [ts for i,ts in enumerate(ts_changes) if i==0 or (i!=0 and ts_changes[i-1]!=ts)]
    return ts_changes

def midi2ts_changes(midi_path, remove_duplicates = True):
    midi = pm.PrettyMIDI(midi_path)
    ts_changes = [(ts.time, str(ts.numerator)+"/"+str(ts.denominator)) for ts in midi.time_signature_changes]
    if remove_duplicates:
    #remove the duplicates
        ts_changes = [ts for i,ts in enumerate(ts_changes) if i==0 or (i!=0 and ts_changes[i-1][1]!=ts[1])]
    return ts_changes

def align_score_ts_with_midi_ts(score_path,midi_path):
    midi_ts_changes = midi2ts_changes(midi_path)
    score_ts_changes = score2ts_changes(score_path)
    out = []
    for si, sts in enumerate(score_ts_changes):
        found_at_index = None
        for mi, mts in enumerate(midi_ts_changes):
            if sts == mts[1]:
                out.append((sts,mts[0]))
                found_at_index = mi
                break
        if found_at_index is None:
            raise(ValueError("The algorithm of ts aligning is not working"))
        if found_at_index< len(midi_ts_changes):
            midi_ts_changes = midi_ts_changes[found_at_index+1:]
    return out

def row_to_ts_align(row):
    try:
        return align_score_ts_with_midi_ts(str(Path(BASE_PATH,row["xml_score"])),str(Path(BASE_PATH,row["midi_score"])))
    except Exception as e:
        print("Processing piece",row["midi_score"], "..................Exception!!" )
        print(e)
        return None 


In [9]:
def align_ts_to_beat(row):
    print("Aligning opus", row.midi_score)
    
    ann_list = list(anns.items())
    downbeats = [b[0] for b in ann_list if b[1]=="db"]
    
    #compute ts_changes
    ts_changes = row_to_ts_align(row)
    # align ts
    aligned_ts = []
    for ts in ts_changes:
        # find beat in the window
        close_db = [db for db in downbeats if (db >= ts[1] - 0.0175) and (db <= ts[1] + 0.0175)]
        if ts[1] == 0: #initial time signature, can not coincide with downbeats, so don't align it
            aligned_ts.append(ts)
        elif len(close_db) != 1: # if no close note found (beat on a rest)
            print("Some problems for ts",ts)
        else:
            aligned_ts.append((ts[0],close_db[0])) # align to the close downbeat
            shift = np.abs(ts[1]- close_db[0])
            if shift > 0:
                print("ts",ts, "moved of ",shift)
    
    # move the first time signature to the first downbeat position
    first_db = downbeats[0]
    first_ts = aligned_ts[0][1]
    if first_db!= first_ts:
        aligned_ts[0] = (aligned_ts[0][0],first_db)

    return aligned_ts


ts_changes = align_ts_to_beat(row)

Aligning opus Schubert/Wanderer_fantasie/midi_score.mid
Some problems for ts ('2/2', 334.221888)
Some problems for ts ('3/4', 862.221888)
Some problems for ts ('4/4', 1115.661888)


In [10]:
def row2file_annotations(row):
    #in case beats are missing, exit
    path = Path(BASE_PATH,row["midi_score_annotations"])
    #produce the txt
    file_content = ""
    for a in list(anns.items()):
        file_content+= str(a[0]) + "\t" + str(a[0]) + "\t" + a[1] 
        #add the ts change if needed
        if a[0] in [ts[1] for ts in ts_changes]:
            file_content+=","+ str([ts[0] for ts in ts_changes if ts[1]==a[0]][0])
        file_content+= "\n"
    with open(path,"w") as f:
        f.write(file_content)
    
row2file_annotations(row)

### Add the key signature

In [11]:
def score2ks_changes(score_path, remove_duplicates = True, enharmonic = True):
    score = m21.converter.parse(score_path)
#     keys = [ ks.sharps for ks in score.parts[0].recurse().getElementsByClass(m21.key.KeySignature)]
    keys = [ (ks.asKey(mode='major').tonic.pitchClass,ks.measureNumber,ks.sharps) for ks in score.parts[0].recurse().getElementsByClass(m21.key.KeySignature)]
    if remove_duplicates:
    #remove the duplicates
        if enharmonic: #we condider only the key as an integer [0,11]
            keys = [ks for i,ks in enumerate(keys) if i==0 or (i!=0 and keys[i-1][0]!=ks[0])]
        else: #we consider the number of sharps and flats
            keys = [ks for i,ks in enumerate(keys) if i==0 or (i!=0 and keys[i-1][2]!=ks[2])]
    return keys

def midi2ks_changes(midi_path, remove_duplicates = True):
    midi = pm.PrettyMIDI(midi_path)
    # 0 is C Major, 12 is C minor.
    keys = [(k.key_number,k.time) for k in midi.key_signature_changes]
    if remove_duplicates:
    #remove the duplicates
        keys = [ks for i,ks in enumerate(keys) if i==0 or (i!=0 and keys[i-1][0]!=ks[0])]
    return keys


def save_key_in_txt_annotations(row):
    #get the keys informations
    score_keys = score2ks_changes(str(Path(BASE_PATH,row["xml_score"])))
    midi_keys = midi2ks_changes(str(Path(BASE_PATH,row["midi_score"])))
#     assert([sk[0] for sk in score_keys]  == [mk[0] for mk in midi_keys]  ) #check if they coincide with the score
    if [sk[0] for sk in score_keys]  != [mk[0] for mk in midi_keys]:
        print("Not working for",row.score_midi,score_keys,midi_keys)
        return

    #get the annotations file
    quant_ann_df = pd.read_csv(Path(BASE_PATH,row["midi_score_annotations"]),header=None, names=["time","time2","type"],sep='\t')
    
    ks_out_dict = {}
    #check if there is a db close to every key change time (except for the first)
    for mk,sk in zip(midi_keys,score_keys):
        if mk[1] == 0:
            # it's the first of the piece, append it to the first annotation
            close_ann = [r["time"] for i,r in quant_ann_df.iterrows()][0]
            ks_out_dict[close_ann] = sk[2]
        else: 
            # find the close beat
#             db_list = [row["time"] for i,row in quant_ann_df.iterrows() if row["type"].split(",")[0]=="b" or ]
            close_anns = [r["time"] for i,r in quant_ann_df.iterrows() if (r["time"] >= mk[1] - 0.0175) and (r["time"] <= mk[1] + 0.0175)]
            if len(close_anns)!=1: 
                # we take the first annotation on the right
                close_ann = [r["time"] for i,r in quant_ann_df.iterrows() if (r["time"] >= mk[1] )][0]
            else:
                close_ann = close_anns[0]
            ks_out_dict[close_ann] = sk[2]
    
    #now write the file
    file_content= ""
    counter_written = 0
    for i, r in quant_ann_df.iterrows():
        if r["time"] in ks_out_dict.keys():
            if len(r["type"].split(","))==1: #no time signature
                file_content+= str(r["time"]) + "\t" + str(r["time"]) + "\t" + str(r["type"]) + ",," + str(ks_out_dict[r["time"]])  + "\n"
            elif len(r["type"].split(","))==2: #time signature
                file_content+= str(r["time"]) + "\t" + str(r["time"]) + "\t" + str(r["type"]) + "," + str(ks_out_dict[r["time"]])  + "\n"
            else:
                raise Exception("Bad annotations type")
        else:
            file_content+= str(r["time"]) + "\t" + str(r["time"]) + "\t" + str(r["type"])+ "\n"
    with open(Path(BASE_PATH,row["midi_score_annotations"]),"w") as f:
        f.write(file_content)
        
    return

In [12]:
save_key_in_txt_annotations(row)

## Clean manually the annotations

Open in Audacity and check

### Check automatically the quality of quantized annotations

In [12]:
#Check early/late db/b
cdq.check_late_early_annot(str(Path(BASE_PATH,row["midi_score"])),str(Path(BASE_PATH,row["midi_score_annotations"])))

#check correct type
cdq.check_annotation_text(str(Path(BASE_PATH,row["midi_score_annotations"])))

#check correct ratio beats downbeats (according to the local time signature)
cdq.check_b_db_ratio(str(Path(BASE_PATH,row["midi_score_annotations"])))

# Create the performances annotations


## Create the Score2Performance mapping with midi2midi alignment algorithm
You need to download and compile the nakamura alignment tool. Then move all the file inside "/util/nak_alignment" and move the file "util/MIDIToMIDIAlign.sh" inside the folder.

In [13]:
NAK_PATH = Path("../util/nak_alignment")

def generate_nak_align(row):
    p = Popen([str(Path(NAK_PATH,"MIDIToMIDIAlign.sh")), str(Path(BASE_PATH,row["midi_score"][:-4])), str(Path(BASE_PATH,row["midi_performance"][:-4]))], stdin=PIPE, stdout=PIPE, stderr=PIPE)
    output, err = p.communicate(b"input data that is passed to subprocess' stdin")
    rc = p.returncode
    print(output,err)

In [20]:
all_df = pd.read_csv(Path(BASE_PATH,"metadata.csv"))
df = all_df[all_df.midi_performance == "Schubert/Impromptu_op.90_D.899/4/Dossin04.mid"].copy()
df.apply(generate_nak_align,axis=1)

b'' b''


988    None
dtype: object

## Create annotations

In [21]:
def cleaned_m2m_df(midi2midi_path):
    # read the nakamura output file and delete useless lines
    m2mdf = pd.read_csv(midi2midi_path,sep="\t",skiprows=1,header=None, index_col=False)
    m2mdf.columns =["alignID", "alignOntime", "alignSitch" ,"alignPitch", "alignOnvel", "refID", "refOntime", "refSitch", "refPitch", "refOnvel","empty"]
    #drop the last column "empty"
    m2mdf = m2mdf.drop(columns=["empty"])
    #drop the rows where the alignOntime is -1 (note not existing in both files midi)
    return m2mdf[m2mdf["alignOntime"] != -1]

def generate_performance_annotations(m2m_df, cl_quant_ann):
    #create the list with the time of all annotations
    quant_ann_time_list = cl_quant_ann["time"].tolist()
    quant_ann_type_list = cl_quant_ann["type"].tolist()
    perf_ann_list = []
    missing_list = []
    for i,at in enumerate(quant_ann_time_list):
        p_ons=list(m2m_df[(m2m_df["refOntime"]>at-0.020) & (m2m_df["refOntime"]<at+0.020)]["alignOntime"]) #40 ms accepted window
        if len(p_ons) == 0: #no note matching the ann position in the performance
            perf_ann_list.append(None)
#             print("missing corrispondency at quant time", at)
            missing_list.append(("missing",at))
        else:
            perf_ann_list.append(np.median(p_ons)) #median against outliers
    
    # clean the values that are smaller than the value before
    for ind,at in enumerate(perf_ann_list[1:]):
        if (ind >0) and (at is not None) and (perf_ann_list[ind-1] is not None) and (at < perf_ann_list[ind-1]) :
#             print("Unexpected value",at,"smaller than the previous",perf_ann_list[ind-1])
            perf_ann_list[ind] = None
            missing_list.append(("smaller",at))
            
    #save the index for the none values
    none_indices = [index for index,value in enumerate(perf_ann_list) if value is None]
     
    #check if there are no None at the end (there should no, because of the manual corrections)
    if perf_ann_list[-1] is None:
        missing_list.append(("None at end",0))
    #interpolate to fill the None values in the middle
    filled_at = pd.Series(perf_ann_list).interpolate().tolist()
    #fill the values at the beginning
    ind = 0
    while math.isnan(filled_at[ind]): #find the number of None at the beginning
        ind +=1
    for i in range(ind)[::-1]: #fill, starting from the later one
        proposed_value = filled_at[i+1] - (filled_at[i+2]-filled_at[i+1])
        if proposed_value<0: #cap at 0
            proposed_value = 0
        filled_at[i] = proposed_value
    
    #associate the type of annotation
    assert(len(quant_ann_type_list) == len(filled_at))
    perd_ann_dict = dict(zip(filled_at, quant_ann_type_list))
    #add the Warning for the None values
    warnings = False
    for val in none_indices:
        perd_ann_dict[filled_at[val]] += "W"
        warnings = True
    
    return perd_ann_dict ,missing_list
    
def beat_downbeat_to_text(annotations,out_path):
    path = Path(out_path)
    #produce the txt
    file_content = ""
    for a in annotations:
        file_content+= str(a) + "\t" + str(a) + "\t" + annotations[a] + "\n"
    with open(out_path,"w") as f:
        f.write(file_content)
        
def annotations_to_audio(midi_path, annotations, out_path):
    midi_data = pm.PrettyMIDI(str(midi_path))
    audio_fs = 6000
    beat = [a for a in annotations if annotations[a][0] == "b"]
    downbeat = [a for a in annotations if annotations[a][0] == "d"]
    midi_audio = midi_data.synthesize(fs=audio_fs)
    audio_beat = librosa.clicks(beat, sr=audio_fs, click_freq=900, length=len(midi_audio))
    audio_downbeat = librosa.clicks(downbeat, sr=audio_fs, click_freq=1200, length=len(midi_audio))
    soundfile.write(out_path,audio_beat + audio_downbeat+ midi_audio,audio_fs)
    

def perform_row_to_annotations(row):
    ann_quant_df = pd.read_csv(Path(BASE_PATH,row["midi_score_annotations"]),header=None, names=["time","time2","type"],sep='\t')
    #load the m2m alignment
    m2m = cleaned_m2m_df(Path(BASE_PATH,row["midi_performance"][:-4]+"_corresp.txt"))
    #generate the annotations
    ann_dict,missing_list = generate_performance_annotations(m2m, ann_quant_df)
    #write the annotations to txt
    beat_downbeat_to_text(ann_dict,Path(BASE_PATH,row["midi_performance"][:-4]+"_annotations.txt"))
    #write the audio
    annotations_to_audio(str(Path(BASE_PATH,row["midi_performance"][:-4]+".mid")),ann_dict,Path(BASE_PATH,row["midi_performance"][:-4] + "_click.wav"))
#         print("done")
    return missing_list

In [22]:
df
df.apply(perform_row_to_annotations,axis=1)

988    [(missing, 0.0), (missing, 0.5), (missing, 1.0...
dtype: object